# Hands-on Machine Learning (ML) and Sequential Learning (SL)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paolodeangelis/AEM/blob/main/2-Hands-on_ML_SL.ipynb)

In [ ]:
%pip install pymatgen==2020.1.28
%pip install matminer==0.6.2
%pip install scikit_learn==0.22.2
%pip install shap==0.38.1

In [ ]:
import numpy as np
import pandas as pd
import shap
import sklearn
from matminer.featurizers import composition as cf
from matminer.featurizers.base import MultipleFeaturizer
from pymatgen import Composition
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
class MyDecorrelator(BaseEstimator, TransformerMixin):
    """Class to be imported in pipeline (below) for dropping the most correlated columns, preventing data leakage."""

    def __init__(self, threshold):
        self.threshold = threshold
        self.correlated_columns = None

    def fit(self, X, y=None):
        correlated_features = set()
        X = pd.DataFrame(X)
        corr_matrix = X.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if (
                    abs(corr_matrix.iloc[i, j]) > self.threshold
                ):  # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    correlated_features.add(colname)
        print(np.shape(pd.DataFrame(X).drop(labels=correlated_features, axis=1)))
        # print(pd.DataFrame(X).drop(labels=correlated_features, axis=1))
        self.correlated_features = correlated_features
        return self

    def transform(self, X, y=None, **kwargs):
        return (pd.DataFrame(X)).drop(labels=self.correlated_features, axis=1)

In [ ]:
def get_compostion(
    c,
):  # Function to get compositions from chemical formula using pymatgen
    try:
        return Composition(c)
    except:  # noqa: E722
        return None


def featurizing(data, property_interest=None):

    # Featurizer
    f = MultipleFeaturizer(
        [
            cf.Stoichiometry(),
            cf.ElementProperty.from_preset("magpie"),
            cf.ValenceOrbital(props=["avg"]),
            cf.IonProperty(fast=True),
        ]
    )

    # Inputs
    data["composition"] = [get_compostion(mat) for mat in data.Components]

    featurized_data = pd.DataFrame(
        f.featurize_many(data["composition"], ignore_errors=True),
        columns=f.feature_labels(),
        index=data["Components"],
    )
    if property_interest:
        featurized_data[property_interest] = data[property_interest].values
    return featurized_data

## Handle data


1.   Import data (composition and target property for each material)
2.   Extract 145 composition based features for each material
3.   Drop rows with NaN
4.   Split data in a training set (80% of the database) and in a testing set (20% of the database)





In [ ]:
data = pd.read_excel(r"Supercon_data_clean.xlsx")  # Import data

In [ ]:
Featurized_data = featurizing(data, "Tc")  # Extract composition based features

In [ ]:
Featurized_data

In [ ]:
Featurized_data = Featurized_data.dropna()

In [ ]:
Featurized_data

In [ ]:
train_df, test_df = train_test_split(
    Featurized_data, test_size=0.2, random_state=0
)  # split data in training set (80% of the database) and testing set (20% of the database)

In [ ]:
train_df

# Train and validate the predictive model



1.   Define a pipeline of actions to be performed over data
2.   Define a grid of hyperparameters to be tuned
3.   Perform a grid search (i.e., try all the possible combinations of hyperparameters) in 5 fold cross validation
4.   Show the performances of the best pipeline by doing predictions over the testing set



In [ ]:
rf = RandomForestRegressor(random_state=0)

pipe = Pipeline(
    [
        (
            "decorrelation",
            MyDecorrelator(0.9),
        ),  # Drop features with correlation above 0.9
        ("threshold", VarianceThreshold(threshold=0)),  # Drop features with no variance
        (
            "feature_selector",
            SelectPercentile(f_regression),
        ),  # Select features in terms of the most relevant for f_regression
        ("rf", rf),  # Train Random Forest
    ],
    verbose=1,
)

In [ ]:
param_grid = {
    "rf__n_estimators": [100, 200],  # Tune the number of estimators
    "rf__max_features": [
        "auto",
        "sqrt",
    ],  # Tune the number of features to consider when looking for the best split
    "feature_selector__percentile": [
        50,
        100,
    ],  # Tune the percentage of features to retain in terms of f_regression score
}
search = GridSearchCV(pipe, param_grid, n_jobs=-1, verbose=1, cv=5)

In [ ]:
search.fit(train_df.iloc[:, :-1], train_df.iloc[:, -1])

In [ ]:
search.best_params_

In [ ]:
test_predictions = search.predict(
    test_df.iloc[:, :-1]
)  # Predicted y over samples of the testing set
test_labels = test_df.iloc[:, -1].values  # True y over samples of the testing set

r2 = sklearn.metrics.r2_score(
    test_labels, test_predictions
)  # coefficient of determination
mae = mean_absolute_error(test_labels, test_predictions)  # mean absolute error
rmse = np.sqrt(
    mean_squared_error(test_labels, test_predictions)
)  # root mean squared error
delta = max(test_labels) - min(test_labels)

import matplotlib.pyplot as plt

plt.figure(figsize=(3, 3), dpi=190)
plt.scatter(test_labels, test_predictions, c="crimson", alpha=0.2)
p1 = max(max(test_predictions), max(test_labels))
p2 = min(min(test_predictions), min(test_labels))
plt.plot([p1, p2], [p1, p2], "b-")
plt.annotate(
    "$R^2$ = %0.3f" % r2,
    xy=(0.02 * delta, 0.95 * delta),
    xytext=(0.02 * delta, 0.95 * delta),
)
plt.annotate(
    "MAE = %0.3f K" % mae,
    xy=(0.02 * delta, 0.85 * delta),
    xytext=(0.02 * delta, 0.85 * delta),
)
plt.annotate(
    "RMSE = %0.3f K" % rmse,
    xy=(0.02 * delta, 0.75 * delta),
    xytext=(0.02 * delta, 0.75 * delta),
)
plt.xlabel("True $T_c$ (K)")
plt.ylabel("Predicted $T_c$ (K)")
plt.show()

# Manual refit
We manually refit the best pipeline over the all training set, in order to isolate the Random Forest model and to compute the SHAP values with the TreeExplainer over the entire testing set. Otherwise, the pipeline would be seen as a black box by SHAP, and we would be compelled to use the agnostic KernelSHAP interpretation algorithm (which is more inefficient).

In [ ]:
X_train = train_df.iloc[:, :-1].loc[
    :, VarianceThreshold(threshold=0).fit(train_df.iloc[:, :-1]).get_support()
]  # Drop features with 0 variance
X_test = test_df[X_train.columns]

In [ ]:
correlated_features = set()
corr_matrix = X_train.corr()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if (
            abs(corr_matrix.iloc[i, j]) > 0.9
        ):  # we are interested in absolute coeff value
            colname = corr_matrix.columns[i]  # getting the name of column
            correlated_features.add(colname)

X_train = X_train.drop(labels=correlated_features, axis=1)  # drop correlated features
X_test = X_test[X_train.columns]
y_train = train_df.iloc[:, -1]
y_test = test_df.iloc[:, -1]

In [ ]:
X_train = X_train.loc[
    :,
    SelectPercentile(f_regression, percentile=100).fit(X_train, y_train).get_support(),
]  # Select the percentile of best features from the best pipeline
X_test = X_test[X_train.columns]

In [ ]:
X_train

In [ ]:
rf = RandomForestRegressor(random_state=0, n_estimators=100, max_features="auto")
rf.fit(X_train, y_train)

In [ ]:
y_predictions = rf.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_predictions)
mae = mean_absolute_error(y_test, y_predictions)
rmse = np.sqrt(mean_squared_error(y_test, y_predictions))

import matplotlib.pyplot as plt

plt.figure(figsize=(3, 3), dpi=190)
plt.scatter(test_labels, y_predictions, c="crimson", alpha=0.2)
p1 = max(max(y_predictions), max(y_test))
p2 = min(min(y_predictions), min(y_test))
plt.plot([p1, p2], [p1, p2], "b-")

plt.annotate(
    "$R^2$ = %0.3f" % r2,
    xy=(0.02 * delta, 0.95 * delta),
    xytext=(0.02 * delta, 0.95 * delta),
)
plt.annotate(
    "MAE = %0.3f K" % mae,
    xy=(0.02 * delta, 0.85 * delta),
    xytext=(0.02 * delta, 0.85 * delta),
)
plt.annotate(
    "RMSE = %0.3f K" % rmse,
    xy=(0.02 * delta, 0.75 * delta),
    xytext=(0.02 * delta, 0.75 * delta),
)
plt.xlabel("True $T_c$ (K)")
plt.ylabel("Predicted $T_c$ (K)")
plt.show()

# Interpretability
Thanks to the TreeSHAP algorithm, we can find the most relevant features, ranking them in terms of importance with respect to the output.

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

In [ ]:
N = np.shape(X_test)[1]
k = 0.75
import matplotlib.pyplot as plt

cumsum = np.cumsum(np.sort(np.mean(abs(shap_values), axis=0))[::-1])
normalized_cumulative = np.cumsum(np.sort(np.mean(abs(shap_values), axis=0))[::-1]) / (
    np.max(np.cumsum(np.sort(np.mean(abs(shap_values), axis=0))[::-1]))
)


fig, ax = plt.subplots(figsize=(3, 3), dpi=190)
ax.plot(np.arange(N), normalized_cumulative)
ax.plot(np.arange(N), k * np.ones(N))
ind_cross1 = np.argmin(
    np.fabs(normalized_cumulative - k * max(normalized_cumulative) * np.ones(N))
)
# plt.yticks(np.array([0, 0.5, 1]))

ax.annotate(
    "%i features" % (ind_cross1 + 1),
    xy=(ind_cross1 + 1, 0.01),
    xytext=(ind_cross1 + 10, 0.2),
    arrowprops=dict(facecolor="black", shrink=0.000005, width=0.1, headwidth=4),
)
ax.annotate(
    "75% of\nthe maximum",
    xy=(90, 0.73),
    xytext=(100, 0.45),
    arrowprops=dict(facecolor="black", shrink=0.0005, width=0.1, headwidth=4),
)
plt.scatter(ind_cross1, normalized_cumulative[ind_cross1], color="orange")
plt.plot(
    (ind_cross1, ind_cross1),
    (normalized_cumulative[ind_cross1], 0),
    color="orange",
    ls=":",
)
plt.ylim(0, 1.04)
plt.xlabel("Features")
plt.ylabel("Normalized\ncumulative importance")
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_test, max_display=15)

In [ ]:
Output_shap = pd.DataFrame(shap_values, index=X_test.index, columns=X_test.columns)
Output_mean_shap = pd.DataFrame(
    abs(Output_shap).describe().loc["mean"]
    / sum(abs(Output_shap).describe().loc["mean"])
).sort_values("mean", ascending=False)

In [ ]:
Output_mean_shap.to_excel(
    "Output_mean_shap.xlsx"
)  # list of the features ranked in terms of importance (importances sum up to 1)

# Sequential Learning
We consider a random subset of 100 rows of the original database; we perform SL starting with the worst 50 materials in this set, in terms of the target property.

With these very same initial conditions, we perform three parallel SL procedures, choosing the next interesting material to be tested with three different acquisition functions: Maximum Expected Improvement (MEI), Maximum Likelihood Improvement (MLI), Maximum Uncertainty (MU).

Since the random forest regressor by lolopy (used to compute $\mu$ and $\sigma$ for each of the materials of the testing set) is not deterministic, at each iteration of SL, given an acquisition function, we repeat the regression, and the consequent choice of the next material, 10 times; we effectively pick the most preferred material. We repeat the procedure for the three acquisition functions in parallel.

We repeat the procedure both with only the relevant features, and with an extended set of features including also non relevant ones.

In [ ]:
%pip install lolopy

In [ ]:
from lolopy.learners import RandomForestRegressor

In [ ]:
def MEI(X: np.ndarray, y: np.ndarray, n_steps: int, T: int) -> int:
    """Acquisition functions MEI.

    Args:
        X (numpy.ndarray): matrix with n rows (number of total materials for
            which doing the SL, in our case 100) and d columns (number of features
            taken into account for the optimization)
        y (

            .ndarray): vector with n rows (target property)
        n_steps (int): number of steps allowed for doing SL (in our case,
            100 total materials - 50 materials in the initial training set
            = maximum 50 steps to find the optimum)
        T (int): number of times the (non-deterministic) regression and
            the consequent choice of the next material is performed

    Returns:
        int: the index of the chosen material
    """

    arr = y
    minima = arr.argsort()[0:50]
    in_train = np.zeros(len(X), dtype=np.bool)
    in_train[minima] = True

    all_inds = set(range(len(y)))
    F = np.zeros(n_steps)
    G = np.zeros(n_steps)
    mei_train = [list(set(np.where(in_train)[0].tolist()))]
    mei_train_inds = []

    for i in tqdm.tqdm(range(n_steps)):
        mei_train_inds = mei_train[-1].copy()
        mei_search_inds = list(all_inds.difference(mei_train_inds))

        mei_selection_index = []
        for j in range(T):
            model.fit(X[mei_train_inds], y[mei_train_inds])
            mei_y_pred_prov = model.predict(X[mei_search_inds])
            mei_selection_index.append(np.argmax(mei_y_pred_prov))

        mei_index_G = max(set(mei_selection_index), key=mei_selection_index.count)
        mei_index = mei_search_inds[mei_index_G]  # Pick the most preferred entry
        mei_train_inds.append(mei_search_inds[mei_index_G])
        mei_train.append(mei_train_inds)
        G[i] = mei_index
        F[i] = mei_train_inds[-1]
        if mei_train_inds[-1] == np.argmax(y):
            break

    return F


def MLI(X: np.ndarray, y: np.ndarray, n_steps: int, T: int) -> int:
    """Acquisition functions MLI.

    Args:
        X (numpy.ndarray): matrix with n rows (number of total materials for
            which doing the SL, in our case 100) and d columns (number of features
            taken into account for the optimization)
        y (numpy.ndarray): vector with n rows (target property)
        n_steps (int): number of steps allowed for doing SL (in our case,
            100 total materials - 50 materials in the initial training set
            = maximum 50 steps to find the optimum)
        T (int): number of times the (non-deterministic) regression and
            the consequent choice of the next material is performed

    Returns:
        int: the index of the chosen material
    """
    arr = y
    minima = arr.argsort()[0:50]
    in_train = np.zeros(len(X), dtype=np.bool)
    in_train[minima] = True

    all_inds = set(range(len(y)))
    K = np.zeros(n_steps)
    L = np.zeros(n_steps)
    mli_train = [list(set(np.where(in_train)[0].tolist()))]
    mli_train_inds = []

    for i in tqdm.tqdm(range(n_steps)):
        mli_train_inds = mli_train[-1].copy()
        mli_search_inds = list(all_inds.difference(mli_train_inds))

        mli_selection_index = []
        for j in range(T):
            model.fit(X[mli_train_inds], y[mli_train_inds])
            mli_y_pred_prov, mli_y_std_prov = model.predict(
                X[mli_search_inds], return_std=True
            )
            mli_selection_index.append(
                np.argmax(
                    np.divide(
                        mli_y_pred_prov - np.max(y[mli_train_inds]), mli_y_std_prov
                    )
                )
            )

        mli_index_L = max(set(mli_selection_index), key=mli_selection_index.count)
        mli_index = mli_search_inds[mli_index_L]  # Pick the most preferred entry

        mli_train_inds.append(mli_search_inds[mli_index_L])
        mli_train.append(mli_train_inds)
        L[i] = mli_index
        K[i] = mli_train_inds[-1]
        if mli_train_inds[-1] == np.argmax(y):
            break

    return K


def MU(X: np.ndarray, y: np.ndarray, n_steps: int, T: int) -> int:
    """Acquisition functions MU.

    Args:
        X (numpy.ndarray): matrix with n rows (number of total materials for
            which doing the SL, in our case 100) and d columns (number of features
            taken into account for the optimization)
        y (numpy.ndarray): vector with n rows (target property)
        n_steps (int): number of steps allowed for doing SL (in our case,
            100 total materials - 50 materials in the initial training set
            = maximum 50 steps to find the optimum)
        T (int): number of times the (non-deterministic) regression and
            the consequent choice of the next material is performed

    Returns:
        int: the index of the chosen material
    """

    arr = y
    minima = arr.argsort()[0:50]
    in_train = np.zeros(len(X), dtype=np.bool)
    in_train[minima] = True

    all_inds = set(range(len(y)))
    R = np.zeros(n_steps)
    S = np.zeros(n_steps)
    mu_train = [list(set(np.where(in_train)[0].tolist()))]
    mu_train_inds = []

    for i in tqdm.tqdm(range(n_steps)):
        mu_train_inds = mu_train[-1].copy()
        mu_search_inds = list(all_inds.difference(mu_train_inds))

        mu_selection_index = []
        for j in range(T):
            model.fit(X[mu_train_inds], y[mu_train_inds])
            mu_y_pred_prov, mu_y_std_prov = model.predict(
                X[mu_search_inds], return_std=True
            )
            mu_selection_index.append(np.argmax(mu_y_std_prov))

        mu_index_R = max(set(mu_selection_index), key=mu_selection_index.count)
        mu_index = mu_search_inds[mu_index_R]

        mu_train_inds.append(mu_search_inds[mu_index_R])
        mu_train.append(mu_train_inds)  # Pick the most preferred entry
        R[i] = mu_index
        S[i] = mu_train_inds[-1]
        if mu_train_inds[-1] == np.argmax(y):
            break

    return S

In [ ]:
def produce_Data_SL(
    Data: pd.DataFrame,
    Output_mean_shap: pd.DataFrame,
    n_relevant: int,
    target_property: str,
) -> tuple:
    """Function to produce datasets for SL starting from the complete database (Featurized_data).

    Args:
        Data (pandas.DataFrame): complete database (Featureized_data)
        Output_mean_shap (pandas.DataFrame): ranking of features used in terms of importance
        n_relevant (int): number of relevant features
        target_property (str): name of the target property in the complete database

    Returns:
        tuple: containing:
            -  pandas.DataFrame: dataset with only the relevant features + the target property
            -  pandas.DataFrame: dataset with relevant features + set of unrelevant features
                (summing up to 30 columns) + the target property
    """

    relevant_features = list(Output_mean_shap.iloc[:n_relevant].index)
    unrelevant_features = list(
        Output_mean_shap.sort_values("mean").iloc[: int(30 - n_relevant)].index
    )
    all_features = relevant_features + unrelevant_features

    relevant_features.append(target_property)
    all_features.append(target_property)

    Data_sampled = Data.sample(
        n=100, random_state=0
    )  # replace the random_state with your id number

    Data_relevant_features = pd.DataFrame(Data_sampled, columns=relevant_features)
    Data_all_features = pd.DataFrame(Data_sampled, columns=all_features)

    return (Data_relevant_features, Data_all_features)

In [ ]:
model = RandomForestRegressor()

In [ ]:
Output_mean_shap = pd.read_excel(r"Output_mean_shap.xlsx", index_col=0)

In [ ]:
Data_relevant_features, Data_all_features = produce_Data_SL(
    Featurized_data, Output_mean_shap, 15, "Tc"
)

In [ ]:
Data_all_features

In [ ]:
import tqdm

#### Sequential learning with relevant features

In [ ]:
MEI_index_relevant = MEI(
    Data_relevant_features.iloc[:, :-1].values,
    Data_relevant_features.iloc[:, -1].values,
    50,
    10,
)

In [ ]:
MLI_index_relevant = MLI(
    Data_relevant_features.iloc[:, :-1].values,
    Data_relevant_features.iloc[:, -1].values,
    50,
    10,
)

In [ ]:
MU_index_relevant = MU(
    Data_relevant_features.iloc[:, :-1].values,
    Data_relevant_features.iloc[:, -1].values,
    50,
    10,
)

In [ ]:
# What is the best tested materials after the first 5 iterations of MLI with only relevant features?
# It is the last one of the following list
Data_relevant_features.iloc[MLI_index_relevant[0:10]].iloc[:, -1].sort_values()

# DO the same thing with all acquisition functions (MEI, MLI, MU) and for both relevant and
# relevant+unrelevant features based datasets

#### Sequential learning with 100 features (relevant + (100-relevant))

In [ ]:
MEI_index_all = MEI(
    Data_all_features.iloc[:, :-1].values, Data_all_features.iloc[:, -1].values, 50, 10
)

In [ ]:
MEI_index_all

In [ ]:
MLI_index_all = MLI(
    Data_all_features.iloc[:, :-1].values, Data_all_features.iloc[:, -1].values, 50, 10
)

In [ ]:
MU_index_all = MU(
    Data_all_features.iloc[:, :-1].values, Data_all_features.iloc[:, -1].values, 50, 10
)

In [ ]:
MU_index_all

In [ ]:
N = 50 / 2  # number of evaluations in random choice
labels = ["MEI", "MLI", "MU"]
relevant = [
    19 / N,
    12 / N,
    8 / N,
]  # replace numbers with the numbers of evaluations performed by SL for MEI, MLI, MU with only relevant features
all = [
    15 / N,
    12 / N,
    31 / N,
]  # replace numbers with the numbers of evaluations performed by SL for MEI, MLI, MU with also unrelevant features

x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 3), dpi=190)

rects1 = ax.bar(x - width / 2, relevant, width, label="15 features")
rects2 = ax.bar(x + width / 2, all, width, label="30 features")
plt.axhline(y=1, color="k", linewidth=1, linestyle="--")


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel("Number of experiments/\n number of random experiments")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.annotate(
    "random choice",
    xy=(1, 1),
    xytext=(1.3, 1.2),
    arrowprops=dict(facecolor="black", width=0.1, headwidth=4),
)


fig.tight_layout()

plt.show()